# Eliminação de Gauss

A cada passo k, procura-se zerar todos os elementos da coluna k abaixo da linha k (fazer ser triangular superior)

### Resolvendo Ax = b

Primeiramente, concatena-se $[A]$ e $[b]$, obtendo-se:

$$
\begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1n} & | & b_1 \\
a_{21} & a_{22} & \cdots & a_{2n} & | & b_2 \\
\vdots & \vdots & \vdots & \vdots & | & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn} & | & b_n \\
\end{bmatrix}
$$

Após isso, realizam-se operações com o objetivo de zerar os elementos da coluna $k$ abaixo da linha $k$.

In [1]:
import numpy as np

In [2]:
def gauss_elimination(A, b):
    C = np.concatenate((A, b.T), axis=1)
    for k in range(C.shape[0]-1):
        m = np.expand_dims(C[k+1:, k] / C[k, k], -1)
        C[k+1:, k:] -= m @ np.expand_dims(C[k, k:], 0)
    return C[:, :-1], C[:, -1]

In [3]:
def solve_gauss(A, b):
    A, b = gauss_elimination(A, b)
    x = np.zeros(A.shape[0])
    for i in range(A.shape[0]-1, -1, -1):
        x[i] = (b[i] - np.dot(x, A[i, :])) / A[i, i]
    return x

In [5]:
A = np.array([[1, 2, 0], [1, 3, 1], [-2, 0, 1]]).astype('float64')
b = np.array([[3, 5, -1]]).astype('float64')
print(gauss_elimination(A, b))
solve_gauss(A, b)

(array([[ 1.,  2.,  0.],
       [ 0.,  1.,  1.],
       [ 0.,  0., -3.]]), array([ 3.,  2., -3.]))


array([1., 1., 1.])

### Relação com a decomposição L.U

Ao final da eliminação de Gauss, se consegue uma matriz superior, que será a **U**.

Ou seja, temos:

$$E_1 \cdot E_2 \cdots E_n \cdot A = U$$

Portanto:

$$L = E_1^{-1} \cdot E_2^{-1} \cdots E_n^{-1}$$

# Eliminação de Gauss com Pivoteamento Parcial

1. A cada passo **K**, antes da etapa de eliminação, encontre *p* $\in$ [k, n] tal que é o max da coluna
2. Permutar as linhas p e k

In [6]:
import numpy as np

In [17]:
# def gauss_elimination(A, b):
#     C = np.concatenate((A, b.T), axis=1)
#     for k in range(C.shape[0]-1):
#         m = np.expand_dims(C[k+1:, k] / C[k, k], -1)
#         C[k+1:, k:] -= m @ np.expand_dims(C[k, k:], 0)
#     return C[:, :-1], C[:, -1]

def gauss_elimination_partial(A, b):
    n = A.shape[0]
    
    C = np.concatenate((A, b.T), axis=1)
    P = np.arange(n)
    for k in range(n-1):
        p = np.argmax(A[k:, k]) + k # A[p, k] is the maximum on array
        if A[p,k] == 0:
            raise Exception('Singular matrix')
        P[[k, p]] = P[[p, k]]
        C[[k, p]] = C[[p, k]] # swap rows
        
        m = np.expand_dims(C[k+1:, k] / C[k, k], -1)
        C[k+1:, k:] -= m @ np.expand_dims(C[k, k:], 0)
    return P, C[:, :-1], C[:, -1]

In [19]:
def solve_gauss_partial(A, b):
    _, A, b = gauss_elimination_partial(A, b)
    x = np.zeros(A.shape[0])
    for i in range(A.shape[0]-1, -1, -1):
        x[i] = (b[i] - np.dot(x, A[i, :])) / A[i, i]
    return x

In [33]:
A = np.array([[1, 2,0], [1, 3,1],[-2, 0,1]]).astype('float64')
b = np.array([[3, 5, -1]]).astype('float64')

print(gauss_elimination_partial(A, b))
print(solve_gauss_partial(A, b))
print(solve_gauss(A, b))

(array([0, 1, 2]), array([[ 1.,  2.,  0.],
       [ 0.,  1.,  1.],
       [ 0.,  0., -3.]]), array([ 3.,  2., -3.]))
[1. 1. 1.]
[1. 1. 1.]


### Como ficaria a decomposição L.U agora que sabemos fazer pivoteamento parcial?

- P.A = L.U, onde P é uma matriz permutação

### Resolvendo Ax = b

$$(PA)x = (LU)x = Pb$$

$$Ly = Pb$$

$$Ux = y$$